# League Table

In [1]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Database connector 
import mysql.connector
from mysql.connector import Error

In [2]:
# Connect to the database
connection = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = '#',
    database = 'arsenaldb'
)

# Function to execute a SQL query 
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Successful")
    except Error as err:
        print(f"Error: '{err}'")

# Read and Run Query
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall() # Reads data from the database without making any changes to it.
        return result
    except Error as err:
        print(f'Error: "{err}"')

In [3]:
# League Table

# Drop Table
execute_query(connection, """ 
DROP TABLE IF EXISTS league_table
""")

# Create Table
league_table = """
CREATE TABLE league_table(
    SeasonID INT NOT NULL PRIMARY KEY,
    Position INT NOT NULL,
    MatchesPlayed	INT NOT NULL,
    Wins INT,
    Draw INT,
    Loss INT,
    GoalsScored INT,
    xG DECIMAL(5,2),
    GoalsAgainst INT,
    xGA DECIMAL(5,2),
    GoalDifference INT,
    Points INT NOT NULL,
    xPTS DECIMAL(5,2)
)
"""
execute_query(connection, league_table)

Successful
Successful


In [3]:
# Assign the seasons accordingly
season_mapping = {
    2023:'2023-2024',
    2022:'2022-2023',
    2021:'2021-2022',
    2020:'2020-2021',
    2019:'2019-2020'
}

# A function to check and adjust the list accordingly and enusre they are all the same length
def check(data):

    # Adjust the team nammes
    cleaned_list = []
    for i in range(len(data)):
        if len(data[i]) == 13:
            clean_data = [data[i][0]] + [' '.join(data[i][1:3])] + data[i][3:]
            cleaned_list.append(clean_data)
        elif len(data[i]) > 13:
            clean_data = [data[i][0]] + [' '.join(data[i][1:4])] + data[i][4:]
            cleaned_list.append(clean_data)
        elif len(data[i]) == 12:
            cleaned_list.append(data[i])

    return cleaned_list

In [4]:
# Create a function that takes the season and scrapes the league table
def table(season):

    driver = webdriver.Chrome()

    page_url = f"https://understat.com/league/EPL/{season}"
    driver.get(page_url) 

    name = driver.find_elements(By.ID, 'league-chemp')[0]
    table = name.text.split('\n')
    
    # Breaks the table into a list of list
    data = [line.split() for line in table] 

    # Checks the length of each list item and sorts accordingly
    cleaned_list = check(data)

    # Transform to a dataframe
    df = pd.DataFrame(cleaned_list)

    # Assign the first row of the list as the column header
    df.columns = df.iloc[0]
    df = df[1:]

    # Filter to show only Arsenal data
    df_arsenal = df[df['Team'] == 'Arsenal']

    # Split columns on '-' and '+'
    split_columns = ['xG', 'xGA', 'xPTS']
    for col in split_columns:
        df_arsenal[col+'_1'] = df_arsenal[col].str.split('[\+\-]', expand=True)[0]
    # Drop original columns
    df_arsenal.drop(columns=split_columns, inplace=True)

    # Rename the column 
    df_arsenal.rename(columns = {'№':'Position',
                        'M':'MatchesPlayed',
                        'W':'Wins',
                        'D':'Draw',
                        'L':'Loss',
                        'G':'GoalsScored',
                        'GA':'GoalsAgainst',
                        'PTS':'Points',
                        'xG_1' : 'xG',
                        'xGA_1' : 'xGA',
                        'xPTS_1' : 'xPTS'}, inplace=True)

    # Change 'Position' to an interger
    df_arsenal['Position'] = df_arsenal['Position'].astype(int)
    
    # Calculate Goal Difference
    df_arsenal['GoalsScored'] = df_arsenal['GoalsScored'].astype(int)
    df_arsenal['GoalsAgainst'] = df_arsenal['GoalsAgainst'].astype(int)

    df_arsenal['GoalDifference'] = df_arsenal['GoalsScored'] - df_arsenal['GoalsAgainst']

    # Add a current season to respective table
    df_arsenal['SeasonName'] = season_mapping[season]

    # Rearrange data
    df_arsenal = df_arsenal.loc[:,['SeasonName','Position','MatchesPlayed','Wins','Draw','Loss','GoalsScored','xG',
                                   'GoalsAgainst','xGA','GoalDifference','Points','xPTS']]

    driver.close()
    
    return df_arsenal

<>:31: SyntaxWarning: invalid escape sequence '\+'
<>:31: SyntaxWarning: invalid escape sequence '\+'
C:\Users\Razak\AppData\Local\Temp\ipykernel_17864\1936700158.py:31: SyntaxWarning: invalid escape sequence '\+'
  df_arsenal[col+'_1'] = df_arsenal[col].str.split('[\+\-]', expand=True)[0]


### Combine all the data

In [5]:
# Creata a function that merges the tables and locs them converts to a list/dict and to insert into database
def db_table(seasons_table, league_table):
    df_table  = pd.merge(seasons_table, league_table, on = 'SeasonName',how = 'inner')
    df_league_table = df_table.loc[:,['SeasonID','Position','MatchesPlayed','Wins','Draw','Loss','GoalsScored',
                    'xG','GoalsAgainst','xGA','GoalDifference','Points','xPTS']]
    
    # Convert to a dictionary
    league_dict = df_league_table.to_dict(orient='index')
    
    return league_dict

In [6]:
# Read the 'season' table data
run_season = """SELECT * 
FROM season;"""
results = read_query(connection, run_season)

# Convert to pandas dataframe
season_columns = ['SeasonID','SeasonName','CompetitionName']
df_season = pd.DataFrame(results, columns = season_columns)
df_season

,SeasonID,SeasonName,CompetitionName
0,1,2019-2020,English Premier League
1,2,2020-2021,English Premier League
2,3,2021-2022,English Premier League
3,4,2022-2023,English Premier League
4,5,2023-2024,English Premier League


In [ ]:
# Previous season and historical data 
df_arsenal_19 = table(2019)
df_arsenal_20 = table(2020)
df_arsenal_21 = table(2021)
df_arsenal_22 = table(2022)
df_arsenal_23 = table(2023)

# Join all the old data together into a table,
previous_season = [df_arsenal_19,df_arsenal_20,df_arsenal_21, df_arsenal_22, df_arsenal_23]
df_league_table = pd.concat(previous_season, ignore_index=True)

Insert into the database

In [10]:
table_dict = db_table(df_season, df_league_table)
sql_command = []

for i in range(len(table_dict)):
    columns = ", ".join(table_dict[i].keys())
    values = ", ".join(str(value) for value in table_dict[i].values())
    sql_query = f'INSERT INTO league_table ({columns}) VALUES ({values})'
    sql_command.append(sql_query)

sql_command

['INSERT INTO league_table (SeasonID, Position, MatchesPlayed, Wins, Draw, Loss, GoalsScored, xG, GoalsAgainst, xGA, GoalDifference, Points, xPTS) VALUES (1, 8, 38, 14, 14, 10, 56, 50.82, 48, 57.25, 8, 56, 50.15)',
 'INSERT INTO league_table (SeasonID, Position, MatchesPlayed, Wins, Draw, Loss, GoalsScored, xG, GoalsAgainst, xGA, GoalDifference, Points, xPTS) VALUES (2, 8, 38, 18, 7, 13, 55, 52.25, 39, 43.23, 16, 61, 58.72)',
 'INSERT INTO league_table (SeasonID, Position, MatchesPlayed, Wins, Draw, Loss, GoalsScored, xG, GoalsAgainst, xGA, GoalDifference, Points, xPTS) VALUES (3, 5, 38, 22, 3, 13, 61, 63.39, 48, 48.39, 13, 69, 64.76)',
 'INSERT INTO league_table (SeasonID, Position, MatchesPlayed, Wins, Draw, Loss, GoalsScored, xG, GoalsAgainst, xGA, GoalDifference, Points, xPTS) VALUES (4, 2, 38, 26, 6, 6, 88, 76.51, 43, 45.16, 45, 84, 72.53)',
 'INSERT INTO league_table (SeasonID, Position, MatchesPlayed, Wins, Draw, Loss, GoalsScored, xG, GoalsAgainst, xGA, GoalDifference, Points, 

In [ ]:
# Execute the query to insert into the table
for query in sql_command:
    try:
        execute_query(connection, query)
    except Error as err:
        print(f"Error: '{err}'")

# Fetch and Update Table with New Data

In [7]:
# Scrape new data
new_data = table(2023)
new_joined = db_table(df_season, new_data)

# Extract the nested dictionary using the key 0
nested_dict = new_joined[0]

# Construct the string to print using list comprehension and str.join()
set_statements = ", ".join([f"{key} = {value}" for key, value in nested_dict.items()])

# Execute Query
update_query = f'UPDATE league_table SET {set_statements} WHERE SeasonID = 5'
execute_query(connection, update_query)

C:\Users\Razak\AppData\Local\Temp\ipykernel_17864\1936700158.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arsenal[col+'_1'] = df_arsenal[col].str.split('[\+\-]', expand=True)[0]
C:\Users\Razak\AppData\Local\Temp\ipykernel_17864\1936700158.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arsenal[col+'_1'] = df_arsenal[col].str.split('[\+\-]', expand=True)[0]
C:\Users\Razak\AppData\Local\Temp\ipykernel_17864\1936700158.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

Successful


In [8]:
# Run Query
run_table = """SELECT * 
FROM league_table;"""
results = read_query(connection, run_table)

# View as dataframe
# Convert to pandas dataframe
league_columns = ['SeasonID','Position','MatchesPlayed','Wins','Draw','Loss','GoalsScored','xG','GoalsAgaints',
                  'xGA','GoalDifference','Points','xPTS']
df_table = pd.DataFrame(results, columns = league_columns)
df_table

,SeasonID,Position,MatchesPlayed,Wins,Draw,Loss,GoalsScored,xG,GoalsAgaints,xGA,GoalDifference,Points,xPTS
0,1,8,38,14,14,10,56,50.82,48,57.25,8,56,50.15
1,2,8,38,18,7,13,55,52.25,39,43.23,16,61,58.72
2,3,5,38,22,3,13,61,63.39,48,48.39,13,69,64.76
3,4,2,38,26,6,6,88,76.51,43,45.16,45,84,72.53
4,5,1,28,20,4,4,70,61.19,24,22.85,46,64,60.26


In [9]:
# Close the connection
connection.close()

- Scrape the new data
- Clean the data
- Connect to the database 
- Insert straight into the database by updating each row with a unique PlayerID and SeasonID for each